In [1]:
import importlib
import subprocess

import Advanced_SQL as asql
import duckdb
import SQL as sql

_ = importlib.reload(asql)
_ = importlib.reload(sql)

Use this cell to test any sql queries. This can be done in one of the following ways:

1. Fill in the `qry` within the """ """.
2. Call the question functions that you have already filled in from the SQL.py and Advanced_SQL.py files. An example of question_1 being called from the SQL.py file is currently commented out.

This cell can be copied if you wish to run multiple queries.


In [3]:
qry = """  
SELECT * FROM customers
"""

# qry = sql.question_1()


with duckdb.connect("database/loan.db") as cursor:
    result_set = cursor.execute(qry).df()

result_set




,CustomerID,Name,Surname,Age,Gender,Income,Region
0,1,prince,linda,24,Female,89097,GT
1,2,shneur,axel,34,Female,62719,EC
2,3,michelle,anisa,48,Female,45999,EasternCape
3,4,eli,zoe,34,Female,48926,NC
4,5,max,virginia,67,Female,80778,MP
...,...,...,...,...,...,...,...
1009,996,tianna,elizabeth,39,Female,81930,WC
1010,997,mariah,januel,65,Female,45704,MP
1011,998,liam,prince,52,Female,84203,MP
1012,999,crystal,luna,71,Female,52113,LP


Should you break/incorrectly update any database tables, the database can be reset by running the following cell


In [37]:
load_script_path = "database/database_load.py"
subprocess.run(["python3", load_script_path])

CompletedProcess(args=['python3', 'database/database_load.py'], returncode=0)

In [5]:
qry = """  
SELECT TRIM(Name) AS Name, TRIM(Surname) AS Surname, CustomerID
FROM customers
WHERE TRIM(CustomerID) IS NOT NULL
AND TRIM(CustomerID) IN (
    SELECT TRIM(CustomerID)
    FROM customers
    GROUP BY TRIM(CustomerID)
    HAVING COUNT(TRIM(CustomerID)) > 1
)
"""

# qry = Q1


with duckdb.connect("database/loan.db") as cursor:
    result_set = cursor.execute(qry).df()

result_set


,Name,Surname,CustomerID
0,grayson,aziza,9
1,grayson,aziza,9
2,aria,malek,14
3,aria,malek,14
4,juliana,hunter,304
5,alina,carter,160
6,eileen,augustus,630
7,nelson,rehan,501
8,henry,noemi,98
9,alexis,amira,957


In [8]:
qry = """  
SELECT TRIM(Name) AS Name, TRIM(Surname) AS Surname, Income
FROM customers
WHERE UPPER(TRIM(Gender)) IN ('FEMALE', 'F')
AND Income IS NOT NULL
ORDER BY Income DESC
"""

# qry = Q2


with duckdb.connect("database/loan.db") as cursor:
    result_set = cursor.execute(qry).df()

result_set

,Name,Surname,Income
0,caden,eshal,89820
1,sadie,victoria,89647
2,amelia,theodore,89641
3,samuel,hailey,89623
4,sydney,alyssa,89603
...,...,...,...
490,stella,noel,40999
491,amber,yadiel,40771
492,matias,reuben,40529
493,eliezer,richard,40513


In [20]:
qry = """  
SELECT TRIM(LoanTerm) AS LoanTerm, 
       (COUNT(CASE WHEN UPPER(TRIM(ApprovalStatus)) = 'APPROVED' THEN 1 END) * 100.0) / COUNT(DISTINCT TRIM(CustomerID)) AS ApprovedLoanPercentage
FROM loans
WHERE TRIM(LoanTerm) IS NOT NULL
AND TRIM(ApprovalStatus) IS NOT NULL
AND TRIM(CustomerID) IS NOT NULL
GROUP BY TRIM(LoanTerm)
ORDER BY LoanTerm ASC
"""

# qry = Q3


with duckdb.connect("database/loan.db") as cursor:
    result_set = cursor.execute(qry).df()

result_set

,LoanTerm,ApprovedLoanPercentage
0,12,48.730964
1,24,48.258706
2,36,46.195652
3,48,54.377880
4,60,47.263682


In [21]:
qry = """  
SELECT TRIM(CustomerClass) AS CustomerClass, 
       COUNT(DISTINCT CustomerID) AS Count
FROM credit
WHERE TRIM(CustomerClass) IS NOT NULL
AND TRIM(CustomerID) IS NOT NULL
GROUP BY TRIM(CustomerClass)
"""

# qry = Q4


with duckdb.connect("database/loan.db") as cursor:
    result_set = cursor.execute(qry).df()

result_set

,CustomerClass,Count
0,A,480
1,B,514
2,A+,6


In [29]:
qry = """
UPDATE credit
SET CustomerClass = 'C'
WHERE CreditScore BETWEEN 600 AND 650
AND CustomerClass IS NOT NULL
AND rowid = (
    SELECT MIN(rowid)
    FROM credit c2
    WHERE credit.CustomerID = c2.CustomerID
);
"""


# qry = Q5


with duckdb.connect("database/loan.db") as cursor:
    result_set = cursor.execute(qry).df()

result_set

,Count
0,271


In [4]:
qry = """
    SELECT 
        TRIM(c.CustomerClass) AS CustomerClass,
        AVG(CAST(cu.Income AS FLOAT)) AS AverageIncome
    FROM 
        (
            SELECT DISTINCT CustomerID, CustomerClass
            FROM credit
            WHERE TRIM(CustomerClass) IS NOT NULL
        ) c
    JOIN 
        (
            SELECT DISTINCT CustomerID, Income
            FROM customers
            WHERE TRIM(Income) ~ '^[0-9]+(\.[0-9]*)?$' -- Ensures Income contains valid numeric data
        ) cu
    ON 
        c.CustomerID = cu.CustomerID
    GROUP BY 
        TRIM(c.CustomerClass)
    ORDER BY 
        CustomerClass ASC;
"""


# qry = Q1 AD


with duckdb.connect("database/loan.db") as cursor:
    result_set = cursor.execute(qry).df()

result_set

,CustomerClass,AverageIncome
0,A,64117.110417
1,A+,76921.000000
2,B,64800.091440


In [7]:
qry = """
SELECT 
    CASE 
        WHEN TRIM(UPPER(c.Region)) IN ('NC', 'NORTHERNCAPE') THEN 'NC'
        WHEN TRIM(UPPER(c.Region)) IN ('GT', 'GAUTENG') THEN 'GT'
        WHEN TRIM(UPPER(c.Region)) IN ('MP', 'MPUMALANGA') THEN 'MP'
        WHEN TRIM(UPPER(c.Region)) IN ('WC', 'WESTERNCAPE') THEN 'WC'
        WHEN TRIM(UPPER(c.Region)) IN ('EC', 'EASTERNCAPE') THEN 'EC'
        WHEN TRIM(UPPER(c.Region)) IN ('NW', 'NORTHWEST') THEN 'NW'
        WHEN TRIM(UPPER(c.Region)) IN ('FS', 'FREESTATE') THEN 'FS'
        WHEN TRIM(UPPER(c.Region)) IN ('KZN', 'KWAZULU-NATAL') THEN 'KZN'
        WHEN TRIM(UPPER(c.Region)) IN ('LP', 'LIMPOPO') THEN 'LP'
        ELSE 'Other'
    END AS Province,
    COUNT(*) AS RejectedApplications
FROM 
    (
        SELECT DISTINCT CustomerID, Region
        FROM customers
        WHERE TRIM(Region) IS NOT NULL
    ) c
JOIN 
    (
        SELECT DISTINCT CustomerID, ApprovalStatus
        FROM loans
        WHERE TRIM(ApprovalStatus) IS NOT NULL
    ) l
ON 
    c.CustomerID = l.CustomerID
WHERE 
    TRIM(UPPER(l.ApprovalStatus)) = 'REJECTED'
GROUP BY 
    Province
ORDER BY 
    Province ASC;


"""


# qry = Q2 AD


with duckdb.connect("database/loan.db") as cursor:
    result_set = cursor.execute(qry).df()

result_set

,Province,RejectedApplications
0,EC,65
1,FS,51
2,GT,65
3,KZN,51
4,LP,55
5,MP,67
6,NC,56
7,NW,50
8,WC,58


In [12]:
qry = """
CREATE TABLE financing AS
WITH DistinctCustomers AS (
    SELECT DISTINCT CustomerID, Income
    FROM customers
    WHERE CustomerID IS NOT NULL AND Income IS NOT NULL
),
DistinctLoans AS (
    SELECT CustomerID, LoanAmount, LoanTerm, InterestRate, ApprovalStatus
    FROM (
        SELECT *, ROW_NUMBER() OVER (PARTITION BY CustomerID ORDER BY LoanAmount DESC) AS row_num
        FROM loans
        WHERE CustomerID IS NOT NULL
    )
    WHERE row_num = 1
),
DistinctCredits AS (
    SELECT CustomerID, CreditScore
    FROM (
        SELECT *, ROW_NUMBER() OVER (PARTITION BY CustomerID ORDER BY CreditScore DESC) AS row_num
        FROM credit
        WHERE CustomerID IS NOT NULL
    )
    WHERE row_num = 1
)
SELECT 
    c.CustomerID,
    c.Income,
    l.LoanAmount,
    l.LoanTerm,
    l.InterestRate,
    l.ApprovalStatus,
    cr.CreditScore
FROM 
    DistinctCustomers c
JOIN 
    DistinctLoans l ON c.CustomerID = l.CustomerID
JOIN 
    DistinctCredits cr ON c.CustomerID = cr.CustomerID;

"""


# qry = Q3 AD


with duckdb.connect("database/loan.db") as cursor:
    result_set = cursor.execute(qry).df()

result_set

,Count
0,1000


In [13]:
qry = """
SELECT * FROM financing LIMIT 10;
"""


# qry = Q4 AD


with duckdb.connect("database/loan.db") as cursor:
    result_set = cursor.execute(qry).df()

result_set

,CustomerID,Income,LoanAmount,LoanTerm,InterestRate,ApprovalStatus,CreditScore
0,7,56049,21327,48,6.68,Rejected,768
1,12,40729,15325,36,8.04,Approved,692
2,19,65077,29282,24,7.77,Approved,651
3,501,64978,24870,12,7.65,Rejected,677
4,48,43949,21088,48,5.93,Rejected,777
5,57,62522,10964,60,7.01,Rejected,637
6,66,60303,18718,60,4.69,Approved,792
7,67,67802,12910,48,5.72,Approved,686
8,68,50624,10505,12,7.61,Approved,755
9,71,70322,16784,12,8.03,Approved,764


In [22]:
qry = """
CREATE TABLE timeline AS
WITH UniqueRepayments AS (
    SELECT DISTINCT 
        RepaymentID,
        CustomerID,
        Amount,
        RepaymentDate,
        TimeZone
    FROM repayments
    WHERE RepaymentID IS NOT NULL
),
AdjustedRepayments AS (
    SELECT 
        ur.CustomerID,
        ur.RepaymentID,
        ur.Amount,
        m.MonthName,
        ur.RepaymentDate,
        -- Adjust repayment time to GMT (London Time)
        CASE 
            WHEN ur.TimeZone = 'JST' THEN ur.RepaymentDate - INTERVAL '9 hours'
            WHEN ur.TimeZone = 'PST' THEN ur.RepaymentDate + INTERVAL '8 hours'
            WHEN ur.TimeZone = 'CET' THEN ur.RepaymentDate - INTERVAL '1 hour'
            WHEN ur.TimeZone = 'EET' THEN ur.RepaymentDate - INTERVAL '2 hours'
            WHEN ur.TimeZone = 'PNT' THEN ur.RepaymentDate + INTERVAL '7 hours'
            WHEN ur.TimeZone = 'CST' THEN ur.RepaymentDate + INTERVAL '6 hours'
            WHEN ur.TimeZone = 'IST' THEN ur.RepaymentDate - INTERVAL '5 hours 30 minutes'
            WHEN ur.TimeZone = 'UTC' THEN ur.RepaymentDate
            WHEN ur.TimeZone = 'GMT' THEN ur.RepaymentDate
            ELSE NULL -- Handle unexpected time zones as NULL
        END AS AdjustedRepaymentDate
    FROM 
        UniqueRepayments ur
    LEFT JOIN 
        months m ON strftime('%m', ur.RepaymentDate) = m.MonthID
),
FilteredRepayments AS (
    SELECT 
        CustomerID,
        MonthName,
        RepaymentID,
        Amount
    FROM 
        AdjustedRepayments
    WHERE 
        AdjustedRepaymentDate IS NOT NULL
        AND CAST(strftime('%H', AdjustedRepaymentDate) AS INTEGER) BETWEEN 6 AND 18 -- Between 6 AM and 6 PM GMT
)
SELECT 
    c.CustomerID,
    m.MonthName,
    COUNT(fr.RepaymentID) AS NumberOfRepayments,
    COALESCE(SUM(fr.Amount), 0) AS AmountTotal
FROM 
    (SELECT DISTINCT CustomerID FROM customers WHERE CustomerID IS NOT NULL) c
CROSS JOIN 
    months m
LEFT JOIN 
    FilteredRepayments fr ON c.CustomerID = fr.CustomerID AND m.MonthName = fr.MonthName
GROUP BY 
    c.CustomerID, m.MonthName
ORDER BY 
    c.CustomerID, m.MonthName;
"""



# qry = Q4 AD


with duckdb.connect("database/loan.db") as cursor:
    result_set = cursor.execute(qry).df()

result_set

,Count
0,12000


In [23]:
qry = """
SELECT * FROM timeline LIMIT 20;
"""


# qry = Q4 AD


with duckdb.connect("database/loan.db") as cursor:
    result_set = cursor.execute(qry).df()

result_set

,CustomerID,MonthName,NumberOfRepayments,AmountTotal
0,1,April,1,479.0
1,1,August,0,0.0
2,1,December,0,0.0
3,1,February,0,0.0
4,1,January,0,0.0
5,1,July,1,451.0
6,1,June,0,0.0
7,1,March,0,0.0
8,1,May,0,0.0
9,1,November,0,0.0


In [26]:
qry = """
SELECT 
    CustomerID,
    -- January
    SUM(CASE WHEN MonthName = 'January' THEN NumberOfRepayments ELSE 0 END) AS JanuaryRepayments,
    SUM(CASE WHEN MonthName = 'January' THEN AmountTotal ELSE 0 END) AS JanuaryTotal,
    -- February
    SUM(CASE WHEN MonthName = 'February' THEN NumberOfRepayments ELSE 0 END) AS FebruaryRepayments,
    SUM(CASE WHEN MonthName = 'February' THEN AmountTotal ELSE 0 END) AS FebruaryTotal,
    -- March
    SUM(CASE WHEN MonthName = 'March' THEN NumberOfRepayments ELSE 0 END) AS MarchRepayments,
    SUM(CASE WHEN MonthName = 'March' THEN AmountTotal ELSE 0 END) AS MarchTotal,
    -- April
    SUM(CASE WHEN MonthName = 'April' THEN NumberOfRepayments ELSE 0 END) AS AprilRepayments,
    SUM(CASE WHEN MonthName = 'April' THEN AmountTotal ELSE 0 END) AS AprilTotal,
    -- May
    SUM(CASE WHEN MonthName = 'May' THEN NumberOfRepayments ELSE 0 END) AS MayRepayments,
    SUM(CASE WHEN MonthName = 'May' THEN AmountTotal ELSE 0 END) AS MayTotal,
    -- June
    SUM(CASE WHEN MonthName = 'June' THEN NumberOfRepayments ELSE 0 END) AS JuneRepayments,
    SUM(CASE WHEN MonthName = 'June' THEN AmountTotal ELSE 0 END) AS JuneTotal,
    -- July
    SUM(CASE WHEN MonthName = 'July' THEN NumberOfRepayments ELSE 0 END) AS JulyRepayments,
    SUM(CASE WHEN MonthName = 'July' THEN AmountTotal ELSE 0 END) AS JulyTotal,
    -- August
    SUM(CASE WHEN MonthName = 'August' THEN NumberOfRepayments ELSE 0 END) AS AugustRepayments,
    SUM(CASE WHEN MonthName = 'August' THEN AmountTotal ELSE 0 END) AS AugustTotal,
    -- September
    SUM(CASE WHEN MonthName = 'September' THEN NumberOfRepayments ELSE 0 END) AS SeptemberRepayments,
    SUM(CASE WHEN MonthName = 'September' THEN AmountTotal ELSE 0 END) AS SeptemberTotal,
    -- October
    SUM(CASE WHEN MonthName = 'October' THEN NumberOfRepayments ELSE 0 END) AS OctoberRepayments,
    SUM(CASE WHEN MonthName = 'October' THEN AmountTotal ELSE 0 END) AS OctoberTotal,
    -- November
    SUM(CASE WHEN MonthName = 'November' THEN NumberOfRepayments ELSE 0 END) AS NovemberRepayments,
    SUM(CASE WHEN MonthName = 'November' THEN AmountTotal ELSE 0 END) AS NovemberTotal,
    -- December
    SUM(CASE WHEN MonthName = 'December' THEN NumberOfRepayments ELSE 0 END) AS DecemberRepayments,
    SUM(CASE WHEN MonthName = 'December' THEN AmountTotal ELSE 0 END) AS DecemberTotal
FROM 
    timeline
GROUP BY 
    CustomerID
ORDER BY 
    CustomerID;
"""


# qry = Q5 AD


with duckdb.connect("database/loan.db") as cursor:
    result_set = cursor.execute(qry).df()

result_set

,CustomerID,JanuaryRepayments,JanuaryTotal,FebruaryRepayments,FebruaryTotal,MarchRepayments,MarchTotal,AprilRepayments,AprilTotal,MayRepayments,...,AugustRepayments,AugustTotal,SeptemberRepayments,SeptemberTotal,OctoberRepayments,OctoberTotal,NovemberRepayments,NovemberTotal,DecemberRepayments,DecemberTotal
0,1,0.0,0.0,0.0,0.0,0.0,0.0,1.0,479.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,1.0,359.0,0.0,0.0,1.0,352.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,58.0
2,3,2.0,413.0,0.0,0.0,1.0,154.0,1.0,236.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,0.0,0.0,0.0,0.0,0.0,0.0,1.0,463.0,0.0,...,0.0,0.0,1.0,123.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,0.0,0.0,0.0,0.0,0.0,0.0,1.0,468.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,996,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
996,997,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
997,998,1.0,479.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,133.0,0.0,0.0
998,999,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,95.0,0.0,0.0,0.0,0.0,0.0,0.0


In [56]:
load_script_path = "database/database_load.py"
subprocess.run(["python3", load_script_path])

qry = """
CREATE TABLE corrected_customers AS
WITH MaleData AS (
    -- Filter male customers and remove duplicates based on CustomerID
    SELECT DISTINCT 
        CustomerID,
        Age,
        Gender
    FROM customers
    WHERE UPPER(TRIM(Gender)) IN ('MALE', 'M')
),
FemaleData AS (
    -- Filter female customers and remove duplicates based on CustomerID
    SELECT DISTINCT 
        CustomerID,
        Age,
        Gender
    FROM customers
    WHERE UPPER(TRIM(Gender)) IN ('FEMALE', 'F')
),

-- Correct ages for males by shifting upwards using LEAD with a 2-row shift
MaleCorrected AS (
    SELECT 
        CustomerID,
        Age,
        -- Adjust the age by shifting 2 rows upwards
        LEAD(Age, 2) OVER (ORDER BY CustomerID) AS CorrectedAge,
        Gender
    FROM MaleData
),
-- Correct ages for females similarly by shifting upwards using LEAD with a 2-row shift
FemaleCorrected AS (
    SELECT 
        CustomerID,
        Age,
        -- Adjust the age by shifting 2 rows upwards
        LEAD(Age, 2) OVER (ORDER BY CustomerID) AS CorrectedAge,
        Gender
    FROM FemaleData
),

-- Handle NaN values in the MaleCorrected table by replacing the last 2 rows with the top 2 rows' ages
MaleFinal AS (
    SELECT 
        CustomerID,
        Age,
        CASE 
            -- Replace the second last row with the first row's age
            WHEN ROW_NUMBER() OVER (ORDER BY CustomerID DESC) = 2 THEN 
                (SELECT Age FROM MaleCorrected ORDER BY CustomerID LIMIT 1) -- First row's Age
            -- Replace the last row with the second row's age
            WHEN ROW_NUMBER() OVER (ORDER BY CustomerID DESC) = 1 THEN 
                (SELECT Age FROM MaleCorrected ORDER BY CustomerID LIMIT 2 OFFSET 1) -- Second row's Age
            ELSE CorrectedAge -- For all other rows, use the corrected age
        END AS CorrectedAge,
        Gender
    FROM MaleCorrected
),
-- Handle NaN values in the FemaleCorrected table by replacing the last 2 rows with the top 2 rows' ages
FemaleFinal AS (
    SELECT 
        CustomerID,
        Age,
        CASE 
            -- Replace the second last row with the first row's age
            WHEN ROW_NUMBER() OVER (ORDER BY CustomerID DESC) = 2 THEN 
                (SELECT Age FROM FemaleCorrected ORDER BY CustomerID LIMIT 1) -- First row's Age
            -- Replace the last row with the second row's age
            WHEN ROW_NUMBER() OVER (ORDER BY CustomerID DESC) = 1 THEN 
                (SELECT Age FROM FemaleCorrected ORDER BY CustomerID LIMIT 2 OFFSET 1) -- Second row's Age
            ELSE CorrectedAge -- For all other rows, use the corrected age
        END AS CorrectedAge,
        Gender
    FROM FemaleCorrected
),

-- Combine the corrected tables for males and females
Combined AS (
    SELECT CustomerID, Age, CorrectedAge, Gender FROM MaleFinal
    UNION ALL
    SELECT CustomerID, Age, CorrectedAge, Gender FROM FemaleFinal
)

-- Final selection of distinct customer IDs, ensuring the correct format for the Gender column
SELECT DISTINCT 
    CustomerID,
    Age,
    CorrectedAge,
    CASE 
        WHEN UPPER(TRIM(Gender)) IN ('FEMALE', 'F') THEN 'Female'
        WHEN UPPER(TRIM(Gender)) IN ('MALE', 'M') THEN 'Male'
        ELSE 'Other' 
    END AS Gender
FROM Combined
ORDER BY CustomerID;

SELECT * FROM corrected_customers;

"""


# qry = Q6 AD


with duckdb.connect("database/loan.db") as cursor:
    result_set = cursor.execute(qry).df()

result_set

,CustomerID,Age,CorrectedAge,Gender
0,1,24,48,Female
1,2,34,34,Female
2,3,48,67,Female
3,4,34,59,Female
4,5,67,18,Female
...,...,...,...,...
995,996,39,52,Female
996,997,65,71,Female
997,998,52,24,Female
998,999,71,34,Female


In [57]:
qry = """
WITH AgeCategoryData AS (
    -- Assigning AgeCategory based on CorrectedAge
    SELECT 
        CustomerID,
        Age,
        CorrectedAge,
        Gender,
        CASE 
            WHEN CorrectedAge < 20 THEN 'Teen'
            WHEN CorrectedAge >= 20 AND CorrectedAge < 30 THEN 'Young Adult'
            WHEN CorrectedAge >= 30 AND CorrectedAge < 60 THEN 'Adult'
            WHEN CorrectedAge >= 60 THEN 'Pensioner'
        END AS AgeCategory
    FROM corrected_customers
),

RepaymentData AS (
    -- Counting repayments per customer
    SELECT 
        CustomerID,
        COUNT(*) AS TotalRepayments
    FROM repayments  -- Assuming a `repayments` table exists where we can count the repayments per customer
    GROUP BY CustomerID
),

RankData AS (
    -- Combining the AgeCategory with repayment count and assigning rank
    SELECT 
        a.CustomerID,
        a.Age,
        a.CorrectedAge,
        a.Gender,
        a.AgeCategory,
        COALESCE(b.TotalRepayments, 0) AS TotalRepayments,
        DENSE_RANK() OVER (PARTITION BY a.AgeCategory ORDER BY COALESCE(b.TotalRepayments, 0) DESC) AS Rank
    FROM AgeCategoryData a
    LEFT JOIN RepaymentData b ON a.CustomerID = b.CustomerID
)

-- Final selection of required columns
SELECT 
    CustomerID,
    Age,
    CorrectedAge,
    Gender,
    AgeCategory,
    Rank
FROM RankData
ORDER BY CustomerID;

"""


# qry = Q7 AD


with duckdb.connect("database/loan.db") as cursor:
    result_set = cursor.execute(qry).df()

result_set

,CustomerID,Age,CorrectedAge,Gender,AgeCategory,Rank
0,1,24,48,Female,Adult,7
1,2,34,34,Female,Adult,8
2,3,48,67,Female,Pensioner,4
3,4,34,59,Female,Adult,7
4,5,67,18,Female,Teen,6
...,...,...,...,...,...,...
995,996,39,52,Female,Adult,8
996,997,65,71,Female,Pensioner,12
997,998,52,24,Female,Young Adult,9
998,999,71,34,Female,Adult,9
